# Data related tasks

## 1.Import libraries


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore') # Ignore warnings

## 2.Read excel
Naming table columns

In [ ]:
cols = ["Kỳ", "Trường_Viện_Khoa", "Mã lớp", "Mã lớp kèm", "Mã HP", "Tên HP", "Tên HP Tiếng Anh", "Khối lượng", "Ghi chú", "Buổi số", "Thứ", "Thời gian", "Bắt đầu", "Kết thúc", "Kíp", "Tuần", "Phòng", "Cần thí nghiệm", "Số lượng đăng ký", "Số lượng max", "Trạng thái", "Loại lớp", "Đợt mở", "Mã quản lý"]
df = pd.read_excel("TKB20231-FULL-1809.xlsx", index_col=0, names=cols)  # name=cols sets the name of each column the specified name in cols array

## 3.Data Filtering

In [ ]:
# Delete title rows
df = df.iloc[2:]
# Create index column, start at 1
df.reset_index(inplace=True, drop=True)
df.index += 1
# filter which column to display
df1 = df[["Mã lớp", "Mã lớp kèm", "Mã HP", "Tên HP", "Khối lượng", "Buổi số", "Thứ", "Thời gian", "Kíp", "Tuần", "Phòng", "Cần thí nghiệm", "Số lượng max", "Trạng thái", "Loại lớp", "Đợt mở", "Mã quản lý"]]
# if "Thí nghiệm", return 1; else return 0
df1["Cần thí nghiệm"] = (df1["Cần thí nghiệm"] == "TN").astype(int)

df1 = df1[df1["Thứ"].notnull()]
# Formating time (0645-0910) to 0645 and 0910
start_time = []
end_time = []
for time in (df1["Thời gian"].to_numpy().tolist()):  # e.g: time = 0645-0910
    start_time.append(time[:4])  # start_time = 0645 -> []
    end_time.append(time[5:])  # end_time = 0910 -> []
df1.insert(loc=7, column="Bắt đầu", value=start_time)  # insert "Bắt đầu" column at column index 7
df1.insert(loc=8, column="Kết thúc", value=end_time)  # insert "Kết thúc" column at column index 8
del df1["Thời gian"]  # We don't need "Thời gian" anymore, so we can delete it
# Group location together
df1["Phòng"] = df1["Phòng"].astype(str)
group_num = []

In [ ]:
def groupingLocation(df1, room):
  """
  Grouping near locations together to reduce calculations
  """
  if any(substring in room for substring in ["D3", "D5"] ): return 0
  # any(substring in room for substring in ["D3", "D5"]) e.g: "D3-5-301" contains D3, therefore the statement is true, applies to other scenarios
  elif any(substring in room for substring in ["D9"]): return 1
  elif any(substring in room for substring in ["C3", "C4", "C5", "C6", "C7", "C8", "C10"]): return 2
  elif any(substring in room for substring in ["C1", "C2", "C9"]): return 3
  elif any(substring in room for substring in ["D4", "D6", "D8"]): return 4
  elif any(substring in room for substring in ["B1", "B3", "B4", "B6", "B7", "B8", "B9", "B10","B13"]): return 5
  elif any(substring in room for substring in ["TC"]): return 6
  else: return 7  # Some places like "Sân vận động" is not included here!!! Possibly have to fix this
for room in (df1["Phòng"].to_numpy().tolist()):
  # df1["Phòng"] type=(dataFrame) --to_numpy()--> type=(numpy array) --tolist()--> type=(array)
  # so df1["Phòng"].to_numpy().tolist() is an array contains rooms of classes
  group_num.append(groupingLocation(df1, room))  # e.g: "D3-5-301" -> 0 -> []
df1.insert(loc=11, column="Khu", value=group_num) # insert "Khu" column at column index 11
del df1["Phòng"] # We don't need "Phòng" anymore, so we can delete it

In [ ]:
df1.head()

,Mã lớp,Mã lớp kèm,Mã HP,Tên HP,Khối lượng,Buổi số,Thứ,Bắt đầu,Kết thúc,Kíp,Tuần,Khu,Cần thí nghiệm,Số lượng max,Trạng thái,Loại lớp,Đợt mở,Mã quản lý
1,738439,738439,BF1601,Sinh học đại cương,3(2-1-1-6),1,5,0730,1145,Sáng,"41,42,43",2,0,18,Điều chỉnh ĐK,TN,B,CT CHUẨN
2,738496,738496,BF1601,Sinh học đại cương,3(2-1-1-6),1,3,0730,1145,Sáng,"16,17,18",2,0,20,Điều chỉnh ĐK,TN,B,CT CHUẨN
3,738497,738497,BF1601,Sinh học đại cương,3(2-1-1-6),1,4,0730,1145,Sáng,"16,17,18",2,0,20,Điều chỉnh ĐK,TN,B,CT CHUẨN
4,738498,738498,BF1601,Sinh học đại cương,3(2-1-1-6),1,5,0730,1145,Sáng,"16,17,18",2,0,20,Điều chỉnh ĐK,TN,B,CT CHUẨN
5,146297,146297,BF2020,Technical Writing and Presentation,3(2-2-0-6),1,3,1230,1550,Chiều,"3-10,12-19",4,0,40,Điều chỉnh ĐK,LT+BT,AB,CT CHUẨN


## 4.Distance Matrix
Used to evaluate the distance between two locations, e.g: `D3` to `TC` is `10`, which is far, students may have to run to get to their next class. While `D3` to `D9` is quite close, the best solution is next class is in the same tower as the previous class (`D3` to `D3` is `0`)

In [ ]:
# Distance matrix
Dist = np.array([[0, 5, 5, 10, 10, 5, 10],  # D3, D5, D3-5 [code 0]
                 [5, 0, 5, 10, 5, 10, 10],  # Library, D9, lake [code 1]
                 [5, 5, 0, 10, 10, 10, 10],  # C6, C7, C8, C3, C4, C5, C10 [code 2]
                 [10, 10, 10, 0, 5, 10, 10],  # C1, C2, C9 [code 3]
                 [10, 5, 10, 5, 0, 10, 10],  # D4, D6, D8 [code 4]
                 [5, 10, 10, 10, 10, 0, 5],  # KTX, B1 [code 5]
                 [10, 10, 10, 10, 10, 5, 0]  # TC =))) [code 6]
                ])

# TODO Some of the places are not on the map such as "Sân vận động", etc so i have grouped them in code 7. Need to fix this ASAP

# Algorithms

## 1.Inputing subjects
Users input subjects. It adds into python dictionary `maHPs` which includes subjects name ID as keys e.g: `IT3011` and possible classes ID as values e.g: `143608`, `144020` and etc (check output for more information)  

In [ ]:
# Dictionary of chosen classIDs
maHPs = {}
maHP = ""
print("Hãy nhập vào mã học phần của môn học bạn định đăng ký: ")
while maHP != "*":
    maHP = input()
    if maHP not in df1["Mã HP"].to_numpy().tolist():
      continue
    if maHP not in maHPs:
        dfx = df1[df1["Mã HP"] == maHP]
        dfx = dfx[dfx["Loại lớp"].isin(["BT", "LT+BT"])]
        filtered_classes_ID = dfx["Mã lớp"].to_numpy().tolist()
        maHPs[maHP] = filtered_classes_ID
print(maHPs)
# Output will only have "BT" or "LT+BT" labeled classes
# Enter "*" to stop

Hãy nhập vào mã học phần của môn học bạn định đăng ký: 
MI3052
MI2020
IT3011
IT3020
*
{'MI3052': ['143854', '144872', '144873', '144874', '144875'], 'MI2020': ['143851', '143852', '144229', '144865', '144866', '144868', '144869', '144890', '144891', '144893', '144894', '145250', '145251', '145256', '145257', '145270', '145271', '145273', '145274', '145285', '145286', '145288', '145289', '145303', '145304', '145483', '145484', '145486', '145487', '145489', '145490'], 'IT3011': ['143608', '143609', '143610', '144020', '144876', '144877', '144903', '144904', '145695', '145696'], 'IT3020': ['143853', '144870', '144870', '144871', '144895', '144896', '144896']}


## 2.Generating initial solution
This will create a possible solution for the problem, but not optimized.
It uses backtracking algorithm to solve. Why not use it to find the optimized solution ? Well, it will have to be looping for a much longer time than the genetic algorithm

In [ ]:
calendar = np.zeros((9,3000)) # it include 7 days from 2-8 and 24 hours from 0000 to 2400. May need to add weeks (19 weeks = 20 layers)
initial_solution = []

def check(class_id, calendar):  # Must turn calendar to previous state after backtracking, maybe creating a temp variable will help
    """
    Checking if the calendar is empty, if empty at the specific classes time -> true, else -> false
    """
    flag = 0
    row = df1["Mã lớp"] == str(class_id)
    date = int(df1.loc[row,"Thứ"])  # Still having errors converting series to int (try >= 6 subjects)
    start = int(df1.loc[row,"Bắt đầu"])
    end = int(df1.loc[row,"Kết thúc"])
    for i in range(start, end+1):  # e.g: study from 0645-0910 -> loop from 645 to 910 and swith them to 1 if empty
        if calendar[date][i] == 0:
            calendar[date][i] = 1
            flag += 1
    if flag < end-start+1: return False # returns 910-645+1 = 266 if flag == this => It is safe to fill this subject in
    return True

def Try(k, tmplist):
    # Backtracking algorithm.
    # Need to fix them to distribute throughout the week, this only solves for first few weekdays
    # Also, add "LT" classes
    """
    Backtracks until the first solution is found
    """
    global initial_solution
    if k == len(maHPs):
        initial_solution = tmplist.copy()
        return
    key = list(maHPs.keys())[k]
    for maHP_loop in maHPs[key]:
        if check(maHP_loop, calendar):
            tmplist.append(maHP_loop)
            Try(k+1, tmplist)
            if initial_solution:
                return
            tmplist.pop()
# Output!!!
Try(0, [])
print("Initial Solution:", initial_solution)

Initial Solution: ['143854', '143851', '143608', '143853']


## 3.Genetic algorithm